In [ ]:
#
# Which two actors are the most similar?
# ... kindof like the "starred most" notebook
# ... but uses networkx to calculate Jaccard similarity
#
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import os
from rtsvg import *
rt = RACETrack()
ofi = rt.ontologyFrameworkInstance(base_filename='../../../data/kaggle_imdb_600k/20240519_ontology')

In [ ]:
#
# 2m31s - amd7900x | edges 1,973,892 | nodes 1,201,864
#

# Just the cast member relationship
df       = ofi.df_triples.filter(pl.col('vrb') == 'castMemberOf')
df_edges = df.group_by(['sbj', 'obj']).len()

# Create a bipartite graph
B = nx.Graph()
B.add_nodes_from(df_edges['sbj'], bipartite=0)
B.add_nodes_from(df_edges['obj'], bipartite=1)
for i in range(len(df_edges)):
    B.add_edge(df_edges['sbj'][i], df_edges['obj'][i])

# Project the graph onto the 'sbj' node set
P = nx.bipartite.projected_graph(B, df_edges['sbj'])

# Calculate Jaccard similarity on the projected graph
similarity = nx.jaccard_coefficient(P)

In [ ]:
# Determine the max score...
# ... didn't finish (or didn't wait long enough) 
max_tuple = None
for x in similarity:
    i, j, sim = x
    if max_tuple is None or sim > max_tuple[2]: max_tuple = (i, j, sim)
print(f'Actor {max_tuple[0]} and {max_tuple[1]} are most similar with a similarity of {max_tuple[2]}')